In [ ]:
import os
import cv2       # conda install -c https://conda.anaconda.org/menpo opencv3
import glob
import ntpath
import numpy
import SimpleITK # conda install -c https://conda.anaconda.org/simpleitk SimpleITK

In [ ]:
extract_mhd_images()

In [ ]:
def extract_mhd_images():
    
    for subset_no in range(0, 10):
    
        src_path = "G:/LungCancerPredict/original/luna_raw/" + "subset" + str(subset_no) + "/"
        src_files = glob.glob(src_path + "*.mhd")
    
        count=1
        
        for src_file in src_files:
            
            print("Counter:",count)
            
            extract_mhd_image(src_file)
            
            count+=1

def extract_mhd_image(src_file):

    patient_id = ntpath.basename(src_file).replace(".mhd", "")

    #创建存储路径
    dst_path = "G:/LungCancerPredict/extracted/luna16_extracted_images/" + patient_id + "/"
    os.makedirs(dst_path, exist_ok=True)
    
    #读取数据
    itk_img = SimpleITK.ReadImage(src_file)
    img_array = SimpleITK.GetArrayFromImage(itk_img)
    print("Original Shape: ", img_array.shape)

    #重采样
    spacing = numpy.array(itk_img.GetSpacing()) 
    img_array = rescale_image(img_array, spacing)
    print("Rescale Shape:",img_array.shape)
   
    for i in range(img_array.shape[0]):
        
        img = img_array[i]
        
        #肺分割
        seg_img, mask = get_segmented_lung(img.copy())
        
        #数据标准化
        org_img = normalize_hu(img)
        
        dst_file = dst_path + "img_" + str(i).rjust(4, '0') + "_i.png"
        cv2.imwrite(dst_file, org_img * 255)
        cv2.imwrite(dst_file.replace("_i.png", "_m.png"), mask * 255)

In [ ]:
def rescale_image(images_zyx, org_spacing_xyz):
    
    target_voxel_mm = 1.00
    resize_z = float(org_spacing_xyz[2]) / float(target_voxel_mm)
    res = cv2.resize(images_zyx, dsize=None, fx=1.0, fy=resize_z, interpolation=cv2.INTER_LINEAR)  # cv2.resize默认数据按照y,x,channel排列
    
    res = res.swapaxes(0, 2)
    res = res.swapaxes(0, 1)
 
    resize_x = float(org_spacing_xyz[0]) / float(target_voxel_mm)
    resize_y = float(org_spacing_xyz[1]) / float(target_voxel_mm)
    res = cv2.resize(res, dsize=None, fx=resize_x, fy=resize_y, interpolation=cv2.INTER_LINEAR)

    res = res.swapaxes(0, 2)
    res = res.swapaxes(2, 1)
   
    return res

from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import disk, binary_erosion, binary_closing
from skimage.filters import roberts
from scipy import ndimage as ndi

def get_segmented_lungs(img):
    
    binary = img < -400                                      # step1: convert to binary
    
    cleared = clear_border(binary)                           # step2: remove blobs connected to the border

    label_image = label(cleared)                             # step3: label the image
    
    areas = [r.area for r in regionprops(label_image)]       # step4: keep the labels with 2 largest areas
    areas.sort()
    
    for region in regionprops(label_image):
        
        if region.area < areas[-2]:
        
            for coordinates in region.coords:
                
                   label_image[coordinates[0], coordinates[1]] = 0
                
    binary = label_image
    
    kernel = disk(2)                                         # step5: erosion operation with a disk of radius 2
    binary = binary_erosion(binary, kernel)
    
    kernel = disk(10)                                        # step6: closure operation with a disk of radius 10
    binary = binary_closing(binary, kernel)                   
    
    edges = roberts(binary)                                  # step7: fill in the small holes inside the binary mask of lungs
    binary = ndi.binary_fill_holes(edges)
    
    get_high_vals = binary == 0                              # step8: superimpose the binary mask on the input image
    img[get_high_vals] = -2000
    
    return img, binary

def normalize_hu(img):
    
    MIN_BOUND = -1000.0
    MAX_BOUND = 400.0
    
    img = (img - MIN_BOUND) / (MAX_BOUND - MIN_BOUND)
    
    img[img > 1] = 1
    img[img < 0] = 0
    
    return img